In [1]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import time
import os
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
WORKSPACE_PATH = '/home/takeyama/pywork/ipython/2016-06-20'

In [3]:
RestorePath = WORKSPACE_PATH+'/raw/'

mozartの環境設定

In [4]:
#p_path="/home/takeyama/pywork/ipython/2016-05-30/"

In [5]:
#cd ~/Documents/SyncRecord/cleaning-addingLABEL/

workstationの環境設定

In [6]:
cd ~/Documents/ALTIMA/20160617-103037/

/home/takeyama/Documents/ALTIMA/20160617-103037


In [7]:
'''
csv_file -> ファイル名
mode Round -> 四捨五入
     Roundup -> 切り上げ
     Rounddown -> 切り捨て
'''
def ImportCSV(csv_file,freq,SensorName,mode='Round'):
        # data dictionary 
        RawData={}   
        
        # design dataframe and import csv
        data = pd.read_csv(csv_file)
        data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        data = data[ data['Type']=='ags']
        
        # convert numpy.darray 
        # Acc Data  [0.1mG]=>[G]
        # Gyr Data  [0.01dps]=>[dps]   ...dps=degree per second
        AccX=data.AccX.values*0.0001
        AccY=data.AccY.values*0.0001
        AccZ=data.AccZ.values*0.0001
        GyrX=data.GyrX.values*0.01
        GyrY=data.GyrY.values*0.01
        GyrZ=data.GyrZ.values*0.01
        
        # regist each raw data 
        RawData['AccX'] = AccX
        RawData['AccY'] = AccY
        RawData['AccZ'] = AccZ
        RawData['GyrX'] = GyrX
        RawData['GyrY'] = GyrY
        RawData['GyrZ'] = GyrZ
        RawData['Name'] = SensorName
        
        # import time by using numpy
        time = data.Time.values #時間の列だけを抽出       
        
        if mode == 'Roundup':
            func = lambda x: int(x/freq)*freq
        elif mode == 'Rounddown':
            func = lambda x: int(x/freq)*freq
        elif mode == 'Round':
            func = lambda x: int((x+freq/2)/freq)*freq
        #ERROR
        else:
            print 'check mode and inputed word is caused error'
            return -1
        
        output = map(func,time)
        RawData['Time'] = np.array(output)

        return RawData
            

In [8]:
def CalcStartTime(array):
    MAX = min(array[0])
    
    for i in range(len(array)):
        if MAX < min(array[i]):
            MAX = min(array[i])
     
    return  MAX

In [9]:
def CalcGoalTime(array):
    MIN = max(array[0])
    
    for i in range(len(array)):
        if MIN > max(array[i]):
            MIN = max(array[i])
     
    return  MIN

**2016-06-14**  
もう１度、前処理について考える。特に、入力データの特徴、出力データの仕様を改める。  
*feacher of Precondition*
1. センサデータは{time, accx, accy, accz, gyrx, gyry, gyrz}が揃っている
2. また、センサデータはすべて整数型となっている。
3. nanデータはない

*output Data*
1. accは[0.1mG]から[G]に単位変換、gyrは[0.01dpg]は[dpg]に変換する。
2. timeはサンプリング周期で丸める。
3. Nanデータは存在する。


In [10]:
"""
    checkData に入れるものは辞書型にする
    freqは計測周期
"""
def NanPating(DicData,freq):
    import time
    start_time = time.time()
    
    # detection for hidden Nan Data
    diffNum =np.array([])
    diffIndex=np.array([])
    checkData = DicData['Time']
    width = len(checkData)
    for i in range(0,width-1):
        if ( checkData[i+1]-checkData[i] )!=freq:
            diffNum=np.append(diffNum, int(checkData[i+1]-checkData[i]) )
            diffIndex=np.append(diffIndex,i)
   
    # insert NAN data to SensorData
    # insert time_data
    def Insert(data,dI,dN,f,mode):
        StartIndex= 0
        tmp =np.array([])
        if mode =='Sensor':
            # insert NAN DATA
            adding = np.nan
            for count,l in enumerate(dI):
                tmp = np.append(tmp, data[StartIndex:int(l)])
                for i in range(0,int(dN[count]/f) ):
                    tmp = np.append(tmp,np.nan)
                StartIndex = int(l)+1
            tmp=np.append(tmp, data[StartIndex:])
        elif mode =='Time':
            # insert 
            for count,l in enumerate(dI):
                tmp = np.append(tmp, data[StartIndex:int(l)])
                for i in range(0,int(dN[count]/f) ):
                    t = int( tmp[-1]+f )
                    tmp = np.append(tmp,t)
                StartIndex = int(l)+1
            tmp=np.append(tmp, data[StartIndex:])
        else:
            print 'mode name error'
        return tmp
    Array ={}
    tmpArrayAccX=Insert(DicData['AccX'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayAccY=Insert(DicData['AccY'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayAccZ=Insert(DicData['AccZ'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayGyrX=Insert(DicData['GyrX'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayGyrY=Insert(DicData['GyrY'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayGyrZ=Insert(DicData['GyrZ'],diffIndex,diffNum,freq,mode='Sensor')
    tmpArrayTime=Insert(DicData['Time'],diffIndex,diffNum,freq,mode='Time')
    Array['AccX'] = tmpArrayAccX
    Array['AccY'] = tmpArrayAccY
    Array['AccZ'] = tmpArrayAccZ
    Array['GyrX'] = tmpArrayGyrX
    Array['GyrY'] = tmpArrayGyrY
    Array['GyrZ'] = tmpArrayGyrZ
    Array['Time'] = tmpArrayTime
    Array['Name'] = DicData['Name']
    #Array=[Time:tmpArrayTime,tmpArrayAccX,tmpArrayAccY,tmpArrayAccZ,tmpArrayGyrX,tmpArrayGyrY,tmpArrayGyrZ]
    elapsed_time = time.time() -start_time
    print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
    return Array

In [11]:
def CalcSearchIndexFromTime(checkData,CompareTime):
    count = 0
    for i in range(0, len(checkData)):
        if CompareTime == checkData[i]:
            print str(CompareTime)+' is much in the index  whose number is '+str(i)
            return i
        

In [12]:
def Extraction(dic,start,goal):
    keys = dic.keys()
    output = {}
    for i in range( len(dic)):
        output[keys[i]] = dic[keys[i]][start:goal]
        if keys[i]=='Time': output[keys[i]] = output[keys[i]].astype(int)
    return output
    

In [13]:
DataCsv1= ImportCSV('14011711-20160617-103037714.csv',1,'sensor1')
DataCsv2= ImportCSV('14011712-20160617-103037714.csv',1,'sensor2')
DataCsv3= ImportCSV('14011713-20160617-103037714.csv',1,'sensor3')
DataCsv4= ImportCSV('14011714-20160617-103037714.csv',1,'sensor4')
DataCsv5= ImportCSV('11411715-20160617-103037714.csv',1,'sensor5')
DataCsv6= ImportCSV('14011716-20160617-103037714.csv',1,'sensor6')
DataCsv7= ImportCSV('14011717-20160617-103037714.csv',1,'sensor7')

In [14]:
def SearchNAN(array,freq):
    diffNum =np.array([])
    diffIndex=np.array([])
    width = len(array)
    for i in range(0,width-1):
        if ( array[i+1]-array[i] )!=freq:
            diffNum=np.append(diffNum, int(array[i+1]-array[i]) )
            diffIndex=np.append(diffIndex,i)
    print 'diffNum='+str(diffNum)
    print 'diffIndex'+str(diffIndex)

In [15]:
MemDataCsv1= ImportCSV('mem/mem-14011711-20160617-103038606.csv',1,'MemSensor1')
MemDataCsv2= ImportCSV('mem/mem-14011712-20160617-103038627.csv',1,'MemSensor2')
MemDataCsv3= ImportCSV('mem/mem-14011713-20160617-103038690.csv',1,'MemSensor3')
MemDataCsv4= ImportCSV('mem/mem-14011714-20160617-103038731.csv',1,'MemSensor4')
MemDataCsv5= ImportCSV('mem/mem-11411715-20160617-103038767.csv',1,'MemSensor5')
MemDataCsv6= ImportCSV('mem/mem-14011716-20160617-103038807.csv',1,'MemSensor6')
MemDataCsv7= ImportCSV('mem/mem-14011717-20160617-103037850.csv',1,'MemSensor7')

In [16]:
print MemDataCsv1['Time'][0]
print MemDataCsv2['Time'][0]
print MemDataCsv3['Time'][0]
print MemDataCsv4['Time'][0]
print MemDataCsv5['Time'][0]
print MemDataCsv6['Time'][0]
print MemDataCsv7['Time'][0]

37838721
37838742
37838805
37838846
37838882
37838922
37837965


In [17]:
print DataCsv1['Time'][0]
print DataCsv2['Time'][0]
print DataCsv3['Time'][0]
print DataCsv4['Time'][0]
print DataCsv5['Time'][0]
print DataCsv6['Time'][0]
print DataCsv7['Time'][0]

37838721
37838742
37838805
37838846
37838882
37838922
37837965


**2016-06-20**  
Altimaで計測して、メモリダウンロードをしたデータを生データとする。  
この生データからウィンドウ幅で切り分けて保存する。


In [18]:
MemTimeArray = [MemDataCsv1['Time'], MemDataCsv2['Time'], MemDataCsv3['Time'], MemDataCsv4['Time'],
               MemDataCsv5['Time'], MemDataCsv6['Time'], MemDataCsv7['Time']]

In [19]:
MemDataArray = [MemDataCsv1, MemDataCsv2, MemDataCsv3, MemDataCsv4, MemDataCsv5, MemDataCsv6, MemDataCsv7]

In [20]:
CalcStartTime(MemTimeArray)

37838922

In [21]:
WINDOW = 16

startTime = CalcStartTime(MemTimeArray)
goalTime = CalcGoalTime(MemTimeArray)

startIndex = np.array([])
goalIndex = np.array([])

for obj in MemTimeArray:
    print 'start'
    startIndex = np.append(startIndex, CalcSearchIndexFromTime(obj, startTime) ).astype(int)
    print 'goal'
    goalIndex = np.append(goalIndex, CalcSearchIndexFromTime(obj, goalTime) ).astype(int)

tmp={}
comMemDataArray = []

for i,memDic in enumerate( MemDataArray ):

    tmp['AccX'] = memDic['AccX'][startIndex[i]:goalIndex[i]]
    tmp['AccY'] = memDic['AccY'][startIndex[i]:goalIndex[i]]
    tmp['AccZ'] = memDic['AccZ'][startIndex[i]:goalIndex[i]]
    tmp['GyrX'] = memDic['GyrX'][startIndex[i]:goalIndex[i]]
    tmp['GyrY'] = memDic['GyrY'][startIndex[i]:goalIndex[i]]
    tmp['GyrZ'] = memDic['GyrZ'][startIndex[i]:goalIndex[i]]
    tmp['Time'] = memDic['Time'][startIndex[i]:goalIndex[i]]
    
    tmp['Name'] = memDic['Name']
    
    if not ( os.path.exists(RestorePath) ):
        os.makedirs(RestorePath)
    
    f = open(RestorePath+tmp['Name']+'.pickle', 'wb')
    pickle.dump(tmp,f)
    comMemDataArray.append(tmp)
    
    print str( tmp['AccX'].shape )

    

start
37838922 is much in the index  whose number is 201
goal
37971170 is much in the index  whose number is 132449
start
37838922 is much in the index  whose number is 180
goal
37971170 is much in the index  whose number is 132428
start
37838922 is much in the index  whose number is 117
goal
37971170 is much in the index  whose number is 132365
start
37838922 is much in the index  whose number is 76
goal
37971170 is much in the index  whose number is 132324
start
37838922 is much in the index  whose number is 40
goal
37971170 is much in the index  whose number is 132288
start
37838922 is much in the index  whose number is 0
goal
37971170 is much in the index  whose number is 132248
start
37838922 is much in the index  whose number is 957
goal
37971170 is much in the index  whose number is 133205
(132248,)
(132248,)
(132248,)
(132248,)
(132248,)
(132248,)
(132248,)


だいたい上記のものをクラスとしてまとめれば良いと思う。  
問題としては柔軟性のあるクラスにしたい。メソッドを選択して前処理を行うようにする。  
すこし構成を考えついてからにする。

In [22]:
len( comMemDataArray )

7

In [23]:
comMemDataArray[0]['AccX'][0]

-0.0115

In [24]:
"""
    dicDataは1つのセンサデータである
    また、各データ（加速度、角速度）はすべて(1234,サンプリング周期)となっていること
"""
def sliding(dicData,axis,samp):
    keys = dicData.keys()
    tmp ={}
    for i in range( 1,len(keys) ):
        tmp[keys[i]] = dicData[keys[i]][axis]

    return tmp

In [25]:
sliding(comMemDataArray[0],1,16)

{'AccX': -0.0117,
 'AccY': -0.0069000000000000008,
 'AccZ': 0.97470000000000001,
 'GyrX': 0.28999999999999998,
 'GyrY': 0.48999999999999999,
 'GyrZ': -0.42999999999999999,
 'Time': 37838923}

In [26]:
"""
    precond は１つのセンサデータ
        例として、（8625,16)の配列
    windowはウィンドウ幅
"""
def CalcFFT(precond,samp):
    tmp = {}
    output = []
    for i in range( len(precond) ):
        w = sliding(precond,i,samp)
        tmp['AccX'] = np.array(fftpack.fft(w['AccX'])[1:(samp/2)+1])
        tmp['AccY'] = np.array(fftpack.fft(w['AccY'])[1:(samp/2)+1])
        tmp['AccZ'] = np.array(fftpack.fft(w['AccZ'])[1:(samp/2)+1])
        tmp['GyrX'] = np.array(fftpack.fft(w['GyrX'])[1:(samp/2)+1])
        tmp['GyrY'] = np.array(fftpack.fft(w['GyrY'])[1:(samp/2)+1])
        tmp['GyrZ'] = np.array(fftpack.fft(w['GyrZ'])[1:(samp/2)+1])
        output.append(tmp)
    return output

**2016-06-21**  
昨日の最後にやっていたプログラムもといコーディング改める

'前提条件'として、メモリダウンロードしたものには欠損がないので

In [27]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import time
import os
import matplotlib.pyplot as plt
import pickle
import copy
%matplotlib inline

1. まずは、計測データのCSVファイルを読み込む

定義  
加速度データを　[0.1mG]=>[G]  
角速度データを　[0.01dps]=>[dps]  


In [28]:
'''
csvFilePath -> ファイル名
mode Round -> 四捨五入
     Roundup -> 切り上げ
     Rounddown -> 切り捨て
'''
def ImportCSV(csvFilePath,freq,sensorName,mode='Round'):
        # data dictionary 
        outPutData={}   
        
        
        # design dataframe and import csv
        df = pd.read_csv(csvFilePath)
        df.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        df = df[ df['Type']=='ags']
        
        # convert numpy.darray 
        # Acc Data  [0.1mG]=>[G]
        mg2g = lambda x : x*0.0001

        # Gyr Data  [0.01dps]=>[dps]   ...dps=degree per second
        mdps2dps = lambda x: x*0.01
        
        AccX= mg2g(df.AccX.values)
        AccY= mg2g(df.AccY.values)
        AccZ= mg2g(df.AccZ.values)
        
        GyrX= mdps2dps(df.GyrX.values)
        GyrY= mdps2dps(df.GyrY.values)
        GyrZ= mdps2dps(df.GyrZ.values)
        
        # regist each raw data 
        outPutData['AccX'] = AccX
        outPutData['AccY'] = AccY
        outPutData['AccZ'] = AccZ
        outPutData['GyrX'] = GyrX
        outPutData['GyrY'] = GyrY
        outPutData['GyrZ'] = GyrZ
        outPutData['Name'] = sensorName
        
        # import time by using numpy
        time = df.Time.values #時間の列だけを抽出       
        
        if mode == 'Roundup':
            _round = lambda x: int(x/freq)*freq
        elif mode == 'Rounddown':
            _round = lambda x: int(x/freq)*freq
        elif mode == 'Round':
            _round = lambda x: int((x+freq/2)/freq)*freq
        #ERROR
        else:
            print 'check mode and inputed word is caused error'
            return -1
        
        output = map(_round,time)
        outPutData['Time'] = np.array(output)

        return outPutData
            

In [29]:
inputData1= ImportCSV('mem/mem-14011711-20160617-103038606.csv',1,'MemSensor1')
inputData2= ImportCSV('mem/mem-14011712-20160617-103038627.csv',1,'MemSensor2')
inputData3= ImportCSV('mem/mem-14011713-20160617-103038690.csv',1,'MemSensor3')
inputData4= ImportCSV('mem/mem-14011714-20160617-103038731.csv',1,'MemSensor4')
inputData5= ImportCSV('mem/mem-11411715-20160617-103038767.csv',1,'MemSensor5')
inputData6= ImportCSV('mem/mem-14011716-20160617-103038807.csv',1,'MemSensor6')
inputData7= ImportCSV('mem/mem-14011717-20160617-103037850.csv',1,'MemSensor7')

2. 共通区間のスタート時間、ゴール時間を求めてから
3. 共通区間を導出する

In [30]:
"""
    dic　辞書型のリスト
"""
def CalcStartTime(dic):
    MAX = min(dic[0]['Time'])
    
    for i in range(len(dic)):
        if MAX < min(dic[i]['Time']):
            MAX = min(dic[i]['Time'])
     
    return  MAX

In [31]:
"""
    dic　辞書型のリスト
"""
def CalcGoalTime(dic):
    MIN = max(dic[0]['Time'])
    
    for i in range(len(dic)):
        if MIN > max(dic[i]['Time']):
            MIN = max(dic[i]['Time'])
     
    return  MIN

In [32]:
"""
    data　辞書型
    keyTime data['Time']の中の探す値
"""
def CalcSearchIndexFromTime(data, keyTime):
    count = 0
    for i in range(0, len(data['Time'])):
        if keyTime == data['Time'][i]:
            print str(keyTime)+' is much in the index  whose number is '+str(i)
            return i

In [33]:
# 保存する場所
RestorePath = WORKSPACE_PATH+'/raw/'

In [34]:
# name name.pickle = reservedName 
def SaveDicDataFromFilePickle(name,data):
    f = open(name+'.pickle', 'wb')
    pickle.dump(data,f)
    f.close()

In [35]:
# name name.pickle = reservedName 
def SaveDicDataFromFileNPZ(name,data):
    np.savez(name, data=data)

In [36]:
def MakeCommonSection(inputDataArray):
    # 共通区間のスタート時間、ゴール時間を求める
    startTime = CalcStartTime(inputDataArray)
    goalTime = CalcGoalTime(inputDataArray)

    # 共通区間のスタート時間のインデックス、ゴール時間のインデックスを探索する
    startIndex = np.array([])
    goalIndex = np.array([])
    for obj in inputDataArray:
        print 'start'
        startIndex = np.append(startIndex, CalcSearchIndexFromTime(obj, startTime) ).astype(int)
        print 'goal'
        goalIndex = np.append(goalIndex, CalcSearchIndexFromTime(obj, goalTime) ).astype(int)

    tmp={}
    comDataArray =[]
    key={}

    # センサデータすべて（時刻、加速度、角速度）に対して共通区間のみのデータを抽出
    for number,iDA in enumerate( inputDataArray ):

        tmp['AccX'] = copy.deepcopy( iDA['AccX'][startIndex[number]:goalIndex[number]] )
        tmp['AccY'] = copy.deepcopy( iDA['AccY'][startIndex[number]:goalIndex[number]] )
        tmp['AccZ'] = copy.deepcopy( iDA['AccZ'][startIndex[number]:goalIndex[number]] )
        tmp['GyrX'] = copy.deepcopy( iDA['GyrX'][startIndex[number]:goalIndex[number]] )
        tmp['GyrY'] = copy.deepcopy( iDA['GyrY'][startIndex[number]:goalIndex[number]] )
        tmp['GyrZ'] = copy.deepcopy( iDA['GyrZ'][startIndex[number]:goalIndex[number]] )
        tmp['Time'] = copy.deepcopy( iDA['Time'][startIndex[number]:goalIndex[number]] )
        tmp['Name'] = copy.deepcopy( iDA['Name'] )
        comDataArray.append(copy.deepcopy(tmp) )    
        key[ tmp['Name'] ] = number
        
        # restore dictionary data
        if not ( os.path.exists(RestorePath) ): os.makedirs(RestorePath)
        SaveDicDataFromFileNPZ(RestorePath+tmp['Name'],tmp)
        
    return key,comDataArray

In [37]:
DataArray=[inputData1, inputData2, inputData3, inputData4, inputData5, inputData6, inputData7 ]

In [38]:
MakeCommonSection(DataArray)

start
37838922 is much in the index  whose number is 201
goal
37971170 is much in the index  whose number is 132449
start
37838922 is much in the index  whose number is 180
goal
37971170 is much in the index  whose number is 132428
start
37838922 is much in the index  whose number is 117
goal
37971170 is much in the index  whose number is 132365
start
37838922 is much in the index  whose number is 76
goal
37971170 is much in the index  whose number is 132324
start
37838922 is much in the index  whose number is 40
goal
37971170 is much in the index  whose number is 132288
start
37838922 is much in the index  whose number is 0
goal
37971170 is much in the index  whose number is 132248
start
37838922 is much in the index  whose number is 957
goal
37971170 is much in the index  whose number is 133205


({'MemSensor1': 0,
  'MemSensor2': 1,
  'MemSensor3': 2,
  'MemSensor4': 3,
  'MemSensor5': 4,
  'MemSensor6': 5,
  'MemSensor7': 6},
 [{'AccX': array([-0.0169, -0.0137, -0.0203, ...,  0.0576,  0.0527,  0.0502]),
   'AccY': array([-0.0102, -0.0158, -0.0175, ...,  0.0316,  0.0308,  0.0328]),
   'AccZ': array([ 0.955 ,  0.9553,  0.9362, ...,  0.9296,  0.9257,  0.9309]),
   'GyrX': array([ 0.7 ,  0.86,  0.8 , ...,  0.45,  0.6 ,  0.6 ]),
   'GyrY': array([-0.45, -0.6 , -0.71, ..., -0.75, -0.74, -0.82]),
   'GyrZ': array([-0.4 , -0.38, -0.4 , ..., -0.48, -0.51, -0.49]),
   'Name': 'MemSensor1',
   'Time': array([37838922, 37838923, 37838924, ..., 37971167, 37971168, 37971169])},
  {'AccX': array([-0.0262, -0.0192, -0.0218, ...,  0.035 ,  0.0328,  0.0358]),
   'AccY': array([-0.01  , -0.0083, -0.0047, ...,  0.039 ,  0.042 ,  0.0378]),
   'AccZ': array([ 0.9787,  0.9792,  0.976 , ...,  0.9557,  0.9545,  0.9557]),
   'GyrX': array([-0.31, -0.43, -0.31, ..., -0.45, -0.59, -0.59]),
   'GyrY': ar

In [39]:
"""
    key dicListの配列番号とdicの名前をつなげる配列
    dicList 辞書型のデータをまとめたリスト
    search 検索するセンサの名前
"""
def GetDicDataFromList(key,dicList,search):
    i = key[search]
    return dicList[i]

In [40]:
"""
    loadName ロードするファイル名
"""
def LoadDicDataFromFilePickle(loadName):
    f = open(loadName+'.pickle')
    dicData = copy.deepcopy( pickle.load(f) )
    f.close()
    return dicData

In [41]:
"""
    loadName ロードするファイル名
"""
def LoadDicDataFromFileNPZ(loadName):
    arrays = np.load(loadName+'.npz')
    output = arrays['data'][()]
    return output

In [42]:
"""
    search 検索するセンサの名前
"""
def LoadDicDataFromFile(search):
    f = open(search+'.pickle')
    dicData = copy.deepcopy( pickle.load(f) )
    f.close()
    return dicData

In [43]:
key, dicList=MakeCommonSection(DataArray)
GetDicDataFromList(key,dicList,'MemSensor5')

start
37838922 is much in the index  whose number is 201
goal
37971170 is much in the index  whose number is 132449
start
37838922 is much in the index  whose number is 180
goal
37971170 is much in the index  whose number is 132428
start
37838922 is much in the index  whose number is 117
goal
37971170 is much in the index  whose number is 132365
start
37838922 is much in the index  whose number is 76
goal
37971170 is much in the index  whose number is 132324
start
37838922 is much in the index  whose number is 40
goal
37971170 is much in the index  whose number is 132288
start
37838922 is much in the index  whose number is 0
goal
37971170 is much in the index  whose number is 132248
start
37838922 is much in the index  whose number is 957
goal
37971170 is much in the index  whose number is 133205


{'AccX': array([-0.0237, -0.0249, -0.0198, ...,  0.0373,  0.0403,  0.041 ]),
 'AccY': array([-0.0059, -0.0023, -0.0018, ...,  0.0461,  0.0426,  0.048 ]),
 'AccZ': array([ 0.9872,  0.9903,  0.9837, ...,  0.972 ,  0.9745,  0.9723]),
 'GyrX': array([ 0.27,  0.13,  0.12, ..., -0.2 , -0.19, -0.17]),
 'GyrY': array([ 2.69, -0.14, -0.92, ...,  0.15,  0.25,  0.36]),
 'GyrZ': array([-0.24, -0.22, -0.13, ..., -0.15, -0.25, -0.38]),
 'Name': 'MemSensor5',
 'Time': array([37838922, 37838923, 37838924, ..., 37971167, 37971168, 37971169])}

In [44]:
dicList[4]

{'AccX': array([-0.0237, -0.0249, -0.0198, ...,  0.0373,  0.0403,  0.041 ]),
 'AccY': array([-0.0059, -0.0023, -0.0018, ...,  0.0461,  0.0426,  0.048 ]),
 'AccZ': array([ 0.9872,  0.9903,  0.9837, ...,  0.972 ,  0.9745,  0.9723]),
 'GyrX': array([ 0.27,  0.13,  0.12, ..., -0.2 , -0.19, -0.17]),
 'GyrY': array([ 2.69, -0.14, -0.92, ...,  0.15,  0.25,  0.36]),
 'GyrZ': array([-0.24, -0.22, -0.13, ..., -0.15, -0.25, -0.38]),
 'Name': 'MemSensor5',
 'Time': array([37838922, 37838923, 37838924, ..., 37971167, 37971168, 37971169])}

In [45]:
LoadDicDataFromFilePickle(RestorePath+'MemSensor5')

{'AccX': array([-0.0237, -0.0249, -0.0198, ...,  0.0373,  0.0403,  0.041 ]),
 'AccY': array([-0.0059, -0.0023, -0.0018, ...,  0.0461,  0.0426,  0.048 ]),
 'AccZ': array([ 0.9872,  0.9903,  0.9837, ...,  0.972 ,  0.9745,  0.9723]),
 'GyrX': array([ 0.27,  0.13,  0.12, ..., -0.2 , -0.19, -0.17]),
 'GyrY': array([ 2.69, -0.14, -0.92, ...,  0.15,  0.25,  0.36]),
 'GyrZ': array([-0.24, -0.22, -0.13, ..., -0.15, -0.25, -0.38]),
 'Name': 'MemSensor5',
 'Time': array([37838922, 37838923, 37838924, ..., 37971167, 37971168, 37971169])}

In [46]:
LoadDicDataFromFileNPZ(RestorePath+'MemSensor5')

{'AccX': array([-0.0237, -0.0249, -0.0198, ...,  0.0373,  0.0403,  0.041 ]),
 'AccY': array([-0.0059, -0.0023, -0.0018, ...,  0.0461,  0.0426,  0.048 ]),
 'AccZ': array([ 0.9872,  0.9903,  0.9837, ...,  0.972 ,  0.9745,  0.9723]),
 'GyrX': array([ 0.27,  0.13,  0.12, ..., -0.2 , -0.19, -0.17]),
 'GyrY': array([ 2.69, -0.14, -0.92, ...,  0.15,  0.25,  0.36]),
 'GyrZ': array([-0.24, -0.22, -0.13, ..., -0.15, -0.25, -0.38]),
 'Name': 'MemSensor5',
 'Time': array([37838922, 37838923, 37838924, ..., 37971167, 37971168, 37971169])}

これから、特徴抽出についてのコーディングを行う  
入力は現在２つ考えている。前処理で生成した変数を渡す、もしくはpickleのデータを読み込む
出力は、今のところ辞書型を考えている。構造は以下のとおりと予定する。
- Sensor1
 - FFT
 - PowerSpectol
- Sensor2
 - FFT
 - PowerSpectol

**2016-06-24**  
今日の目標は特徴ベクトルつまり、Kerasにデータを渡すためのデータの形を考え決める
そして、実装することである。  
午前中に計測したデータをAutoEncoderにかけたい。windowに変換するのはできるがそこから
どのようにしたらいいのかがわからない。　　
対策としては、データの構造を固定できないのであれば逆の発送でどのようなデータ構造にも
柔軟に対応できるようにすればいいかもしれない。

食わすケースを考えてみる　　
1. １つのセンサデータをすべて渡す
2. 2以上のセンサデータをすべて渡す
3. あるセンサの加速度、もしくは角速度のみを渡す
4. 3の複数形
5. あるセンサのFFT、Powerのすべてをわたす
6. 5の複数形
7. あるセンサの生データ、FFT、Powerわたす

In [50]:
# １つのセンサのデータをautoencderにくわす
seed = []
seed.append(MemDataCsv1['AccX'])
seed.append(MemDataCsv1['AccY'])
seed.append(MemDataCsv1['AccZ'])
seed.append(MemDataCsv1['GyrX'])
seed.append(MemDataCsv1['GyrY'])
seed.append(MemDataCsv1['GyrZ'])
seed

[array([ 0.0043,  0.0021, -0.0027, ...,  0.0461,  0.0485,  0.0458]),
 array([-0.0268, -0.0263, -0.0342, ...,  0.0377,  0.0377,  0.0369]),
 array([ 0.9306,  0.8986,  0.9028, ...,  0.9235,  0.9187,  0.9243]),
 array([ 1.11,  0.91,  0.92, ...,  0.44,  0.39,  0.44]),
 array([-0.6 , -0.57, -0.69, ..., -0.92, -0.89, -0.79]),
 array([-0.31, -0.31, -0.34, ..., -0.32, -0.4 , -0.55])]